In [2]:
#필수 라이브러리를 호출합니다.
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
#파이썬 한국어 자연어처리 라이브러리인 KoNLPy의 Mecab을 불러옵니다.

from konlpy.tag import Mecab

#KoNLPy에 대한 자세한 내용은 다음 링크를 확인해주세요(https://konlpy-ko.readthedocs.io/ko/v0.4.3/)
#Mecab에 대한 자세한 설명은 다음 링크를 확인해주세요(http://eunjeon.blogspot.com/2013/02/blog-post.html)
#현재 konlpy에서 메캅은 윈도우즈 환경에서 구동할 수 없습니다. linux 또는 Mac OS를 써주세요.

mecab = Mecab(dicpath='@@@') 
#메캅의 사용자사전을 불러옵니다. dicpath 이하 @@@은 해당 사전의 경로입니다. 앞으로 모든 @@@은 해당 파일이 있는 경로를 의미합니다.

ModuleNotFoundError: No module named 'konlpy'

In [ ]:
#분석용 파일을 불러옵니다. 우선은 범여_편집본2.csv를 보겠습니다.
#편집본은 현행 사전기반 자연어처리의 한계를 극복하기 위해 텍스트 원본의 일부를 수정한 것입니다.(1차 전처리))
#예컨대 기존 사전에 없는 '자유한국당'이라는 단어는 '자유', '한국', '당'으로 처리됩니다. 
#이를 방지하기 위해 '자유한국당'(과 동의어인 '한국당')은 일괄 '새누리당'으로 변경했습니다.(그럼에도 '누리'만 남아서 실 분석에서는 '누리'를 '자유한국당'으로 변경했습니다.)
df7 = pd.read_csv('@@@')
print(df7.shape)#총 몇 줄인지 알아봅니다.
print(df7.content.str.len().sum())#총 몇자인지 알아봅니다.
df7.head()#맨 처음 다섯문장을 보여줍니다.

#위와 같이, 지난 필리버스터에서 범여권은 38만여 음절을 말했습니다.

In [ ]:
#혹시 있을수 있는 NaN value를 없애줍니다.
df7 = df7.fillna('')

In [ ]:
#본격적으로 단어를 추출합니다.
morphs = df7.content.apply(mecab.nouns)#명사만 추출합니다. 
morphs.head()

In [ ]:
#2차 전처리를 진행합니다.
vocab = defaultdict(int)
for words in morphs:
    for word in words:
        if word.isnumeric():
            vocab['NUMBER'] += 1        # 숫자는 `NUMBER`로 대체하고,
        elif re.search(r'^\W+$', word): 
            pass                        #물음표 등 기호는 제거합니다.
        else:
            vocab[word] += 1            #2차 전처리된 단어들을 vocab에 입력합니다.

vocab                                   #vocab을 보여줍니다.

In [ ]:
#저렇게 보긴 어려우니까, 빈도 기준 내림차순을 진행하겠습니다.

pd.DataFrame.from_dict(vocab, 
                       orient='index', 
                       columns=['빈도']).sort_values('빈도', ascending=False).head(20)

In [ ]:
#제외어stop words를 지정합니다,
stop_words=['우리','여러분','사람','나라',
            '의원','국회의원',
            '얘기','이것','말씀','그것','다음','때문','이게','하나','겁니다','생각',
            '여기','어디','이후','이야기','경우','자신','가지','그때','저희','당시',
            '겁니까','이번','거기','누','이상','내년','뭡니까','마찬가지','누구',
            '이름','무엇','대','나중','여','당신','뭔지','지난해','일부','얼마','국',
            '의','한마디'
           ]

#동의어thesaurus를 지정합니다. (딕셔너리로 지정)
thes = {
    '수처':'공수처',
    '비례제':'비례대표제',
    '비례':'비례대표제',
    '비례대표':'비례대표제',
    '필리':'필리버스터',
    '버스터':'필리버스터',
    '여당':'민주당',
    '더민주':'민주당',
    '국당':'한국당',
    '한국당':'자유한국당',
    '새누리당':'자유한국당',
    '문재':'문재인',
    '패스트':'패스트트랙',
    '트랙':'패스트트랙',
}

In [ ]:
#위에서 세팅한 유의어, 제외어를 적용합니다.
reform = []
#제외어 처리
for w in vocab:
    if w not in stop_words:
        reform.append(w)
#유의어 처리
for idx, word in enumerate(reform):
    keys = thes.keys()
    if word in keys:
        reform[idx] = thes[word]
reform

In [ ]:
# reform 기준으로 문서 morphs 정제
morphs = morphs.apply(lambda row: [w for w in row if w in reform])
morphs

#완성된 morph를 기준으로 단어들을 벡터화 합니다.
#벡터화는 컴퓨터가 인간의 언어를 이해하게 하기 위한 필수적인 과정입니다. 
#자세한 내용은 링크를 확인해주세요(https://lovit.github.io/nlp/2018/03/26/from_text_to_matrix/)
count_model = CountVectorizer(max_features=200, stop_words=stop_words) #상위 200단어만 불러옵니다.

In [ ]:
#앞서 불러온 sklearn의 countvectorizer에 입력하기 위하여, 따로 떼어져 있는 단어들을 다시 붙여줍니다.
data = morphs.apply(' '.join)
type(data)

In [ ]:
#벡터라이징 합니다.
vect = count_model.fit_transform(data)

In [ ]:
#의미연결망분석은 결국 단어들 간의 동시출현빈도표를 시각화한 것입니다. 
#지금까지의 전처리와 벡터화 역시 동시출현빈도표를 만들기 위한 과정이었습니다.

mat = (vect.T * vect) #200x200 짜리 매트릭스를 만듭니다.
mat.setdiag(0) # 같은 단어의 동시출현빈도는 0으로 일괄 치환합니다.
print(mat.todense()) # 매트릭스를 dense포맷으로 가져옵니다
#이 코드는 여기서 가져왔습니다 : http://www.pingshiuanchua.com/blog/post/keyword-network-analysis-with-python-and-gephi